In [1]:
from lightning import Fabric
import numpy as np
from sklearn.linear_model import SGDRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from causalpruner.sgd_pruner import ParamDataset

torch.set_float32_matmul_precision('medium')

In [2]:
fb = Fabric()
fb.launch()

In [132]:
ds = ParamDataset("./checkpoints/lenet_cifar10_causalpruner_10_10_1_0_0.9/weights/3/",
                  "./checkpoints/lenet_cifar10_causalpruner_10_10_1_0_0.9/loss/3/")
# dl = DataLoader(ds, batch_size=16)
dl = DataLoader(ds, batch_size=len(ds))
dl = fb.setup_dataloaders(dl)

In [141]:
torch.min(ds.weights_zstats.std)

tensor(0.)

In [133]:
X, Y = next(iter(dl))

In [134]:
X_abs = torch.abs(X)
Y_abs = torch.abs(Y)

print(f"{torch.max(X_abs)}; {torch.min(X_abs)}")
print(f"{torch.max(Y_abs)}; {torch.min(Y_abs)}")

inf; 0.0
3.746915340423584; 0.002070333808660507


In [127]:
X = X.cpu().numpy()
Y = np.ravel(Y.cpu().numpy())

In [136]:
num_params = 11164352

In [131]:
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
Y_mean = np.mean(Y)
Y_std = np.std(Y)

In [141]:
X_new = (X - X_mean) / (X_std * np.sqrt(num_params))
Y_new = (Y - Y_mean) / Y_std

In [142]:
X_new

array([[-1.76475340e-04, -1.92373816e-04, -1.90137129e-04, ...,
        -1.16685937e-04, -8.89538424e-05, -5.31744736e-05],
       [ 2.47291522e-03,  2.06345273e-03,  1.89813389e-03, ...,
         2.65809125e-04, -1.76036017e-04, -2.04287557e-04],
       [ 2.48669367e-05, -1.22674937e-05, -4.20605511e-06, ...,
        -2.06190918e-04, -1.79182854e-04, -2.04287557e-04],
       ...,
       [-5.61698434e-05, -1.09705215e-04, -6.37451376e-05, ...,
         2.94695463e-04,  2.26653065e-04, -1.38175586e-04],
       [-1.73442604e-04, -1.93236803e-04, -1.98398237e-04, ...,
         2.39037636e-05, -1.56191833e-04, -1.06890453e-04],
       [ 1.54735264e-03,  1.69958174e-03,  1.92982843e-03, ...,
         2.94695463e-04,  2.58459506e-04, -1.26369865e-04]], dtype=float32)

In [143]:
trainer = SGDRegressor(loss="squared_error", penalty="l1", alpha=1e-3, fit_intercept=False, shuffle=True, verbose=True)

In [144]:
trainer.fit(X_new, Y)
print(trainer.n_iter_)

-- Epoch 1
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 196, Avg. loss: 0.000072
Total training time: 12.59 seconds.
-- Epoch 2
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 392, Avg. loss: 0.000072
Total training time: 25.11 seconds.
-- Epoch 3
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 588, Avg. loss: 0.000072
Total training time: 37.59 seconds.
-- Epoch 4
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 784, Avg. loss: 0.000072
Total training time: 50.08 seconds.
-- Epoch 5
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 980, Avg. loss: 0.000072
Total training time: 62.61 seconds.
-- Epoch 6
Norm: 0.00, NNZs: 0, Bias: 0.000000, T: 1176, Avg. loss: 0.000072
Total training time: 75.10 seconds.
Convergence after 6 epochs took 75.11 seconds
6


In [95]:
trainer.coef_

array([0., 0., 0., ..., 0., 0., 0.])

In [105]:
dl = DataLoader(ds, batch_size=16, shuffle=True)
dl = fb.setup_dataloaders(dl)

In [106]:
num_params = ds.weights_zstats.num_params
l1_regularization_coeff = 0
lr = 1e-3

In [107]:
model = nn.Linear(num_params, 1, bias=False)
nn.init.zeros_(model.weight)
abs_weight = torch.abs(model.weight)
print(f"{torch.max(abs_weight)}; {torch.min(abs_weight)}")
optimizer = optim.SGD(model.parameters(), lr=lr)
model, optimizer = fb.setup(model, optimizer)

0.0; 0.0


In [108]:
ds.weights_zstats

ZStats(num_params=656500, mean=tensor([2.1518e-09, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 3.2475e-11,
        1.3201e-12]), std=tensor([3.5973e-09, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 4.5777e-11,
        2.0609e-12]))

In [109]:
ds.loss_zstats

ZStats(num_params=1, mean=tensor(-0.0111), std=tensor(0.0030))

In [110]:
# dl = DataLoader(ds, batch_size=len(ds))
# dl = fb.setup_dataloaders(dl)
# X, Y = next(iter(dl))
# X = X.to('cuda')
# Y = Y.to('cuda')

In [111]:
# torch.min(X[torch.isfinite(X)])

In [112]:
# X1 = X[0]
# Y1 = model.weight[0]

# total = 0
# for x, y in zip(X1, Y1):
#     x = torch.nan_to_num(x)
#     y = torch.nan_to_num(y)
#     result = x * y
#     if np.isnan(result.detach().cpu().numpy()):
#         print(x, y, result)
#     total += result
# print(total)

In [113]:
# X_mean_cuda = torch.tensor(X_mean, device='cuda')
# X_std_cuda = torch.tensor(X_std, device='cuda')
# Y_mean_cuda = torch.tensor(Y_mean, device='cuda')
# Y_std_cuda = torch.tensor(Y_std, device='cuda')

In [114]:
num_epochs = int(np.ceil(np.log(num_params / len(ds))))
print(num_epochs)

9


In [128]:
model = nn.Linear(num_params, 1, bias=False)
nn.init.zeros_(model.weight)
abs_weight = torch.abs(model.weight)
print(f"{torch.max(abs_weight)}; {torch.min(abs_weight)}")
optimizer = optim.SGD(model.parameters(), lr=lr)
model, optimizer = fb.setup(model, optimizer)
model.train()
dl = DataLoader(ds, batch_size=16, shuffle=True)
dl = fb.setup_dataloaders(dl)
dl_iter = iter(dl)

0.0; 0.0


In [131]:
X, Y = next(dl_iter)
X = torch.nan_to_num(X, nan=0, posinf=10**10, neginf=-10**10)
Y = torch.nan_to_num(Y, nan=0, posinf=10**10, neginf=-10**10)
optimizer.zero_grad(set_to_none=True)
print(f"{torch.any(torch.isnan(X))}; {torch.any(torch.isinf(X))}")
print(f"{torch.any(torch.isnan(Y))}; {torch.any(torch.isinf(Y))}")
outputs = model(X)
print(f"{torch.any(torch.isnan(outputs))}; {torch.any(torch.isinf(outputs))}")
Y = Y.view(outputs.size())
loss = F.mse_loss(outputs, Y, reduction="mean")
fb.backward(loss)
optimizer.step()
print(f"{torch.any(torch.isnan(model.weight))}; {torch.any(torch.isinf(model.weight))}")

False; False
False; False
False; False
False; True


In [124]:
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 0
    for idx, (X, Y) in enumerate(dl):
        optimizer.zero_grad(set_to_none=True)
        X, Y = torch.nan_to_num(X), torch.nan_to_num(Y)
        print(f"{torch.any(torch.isnan(X))}; {torch.any(torch.isinf(X))}")
        outputs = model(X)
        print(f"{torch.any(torch.isnan(outputs))}; {torch.any(torch.isinf(outputs))}")
        Y = Y.view(outputs.size())
        loss = F.mse_loss(outputs, Y, reduction="mean")
        total_loss += loss.item()
        num_batches += 1
        print(f"epoch: {epoch + 1}; batch: {idx + 1}; Loss: {loss.item()}")
        fb.backward(loss)
        optimizer.step()
        print(f"{torch.any(torch.isnan(model.weight))}; {torch.any(torch.isinf(model.weight))}")
    avg_loss = total_loss / num_batches
    print(f"epoch: {epoch + 1}; avg_loss: {avg_loss}")

False; False
False; True
epoch: 1; batch: 1; Loss: inf
True; True
False; False
True; False
epoch: 1; batch: 2; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 3; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 4; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 5; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 6; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 7; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 8; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 9; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 10; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 11; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 12; Loss: nan
True; False
False; False
True; False
epoch: 1; batch: 13; Loss: nan
True; False
epoch: 1; avg_loss: nan
False; False
True; False
epoch: 2; batch: 1; Loss: nan
True; False
False; False
True; False
epoch: 2; 

In [ ]:
model.weight

In [158]:
torch.max(torch.abs(model.weight))

tensor(0.0525, device='cuda:0', grad_fn=<MaxBackward1>)

In [159]:
torch.min(torch.abs(model.weight))

tensor(1.4300e-09, device='cuda:0', grad_fn=<MinBackward1>)

In [203]:
a = torch.div(0, 0)

In [204]:
a[torch.isnan(a)] = 0

In [205]:
a

tensor(0.)